# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Load dotenv variables
%load_ext dotenv
%dotenv 

# Import necessary modules
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))

# Import logger utility
from utils.logger import get_logger
_logs = get_logger(__name__)


In [2]:
import dask.dataframe as dd

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
from glob import glob

# Use the environment variable to get the path to the parquet files
parquet_files = glob(os.path.join(os.getenv("PRICE_DATA"), "**/*.parquet"), recursive=True)
# Read parquet files into a Dask DataFrame and set the index to 'ticker'
dd_pindex = dd.read_parquet(parquet_files).set_index("ticker")
# Check number of files found
len(parquet_files)


3120

In [4]:

# Load and read the parquet files using Dask
dd_pindex


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=90,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32
ACB,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...


In [5]:
# Check data types of the data on the DataFrame
dd_pindex.dtypes

Date          datetime64[ns]
Open                 float64
High                 float64
Low                  float64
Close                float64
Adj Close            float64
Volume               float64
source       string[pyarrow]
Year                   int32
dtype: object

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
# Create a new column 'Close_lag_1' that contains the previous day's closing price for each ticker
dd_shift = dd_pindex.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

/var/folders/6h/q4ysnrk935x4j8pjrr_90hg00000gn/T/ipykernel_84411/949549128.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_pindex.groupby('ticker', group_keys=False).apply(


In [7]:
dd_feat = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

In [8]:
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Returns
npartitions=90,,,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64
ACB,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...


In [9]:
dd_feat = dd_feat.assign(
    hi_lo_range = lambda x: x["High"] - x["Low"]
)

In [10]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Returns,hi_lo_range
ticker,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,7.153078
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,-0.082386,2.280043
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,0.089783,2.816525
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,-0.090909,2.592991
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,0.026563,1.385908
...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2003-06-26,3.820000,3.830000,3.820000,3.820000,3.498636,7700.0,ZEUS.csv,2003,3.850000,-0.007792,0.010000
ZEUS,2003-06-27,3.820000,3.820000,3.820000,3.820000,3.498636,0.0,ZEUS.csv,2003,3.820000,0.000000,0.000000
ZEUS,2003-06-30,3.830000,4.150000,3.830000,4.090000,3.745922,1700.0,ZEUS.csv,2003,3.820000,0.070681,0.320000


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
import pandas as pd

# Convert Dask DataFrame to Pandas DataFrame for further processing
df_pandas = dd_feat.compute()

# Sort again by ticker and date to prepare for rolling calculations
# Better to sort before rolling calculations (not sure but i thought it better to do)
df_pandas= df_pandas.sort_values(by=["ticker", "Date"])

# Add moving average of returns over a 10-day window
df_pandas["returns_ma_10"] = (
    df_pandas.groupby("ticker")["Returns"]
    .transform(lambda x: x.rolling(10).mean())
)

In [14]:
df_pandas

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Returns,hi_lo_range,returns_ma_10
ticker,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,68.919998,-0.543332,7.153078,NaN
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,-0.082386,2.280043,NaN
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,0.089783,2.816525,NaN
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,-0.090909,2.592991,NaN
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,0.026563,1.385908,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2020-03-26,9.610000,9.940000,9.260000,9.590000,9.590000,60500.0,ZEUS.csv,2020,9.670000,-0.008273,0.679999,0.001594
ZEUS,2020-03-27,9.330000,9.330000,8.700000,8.700000,8.700000,52900.0,ZEUS.csv,2020,9.590000,-0.092805,0.630000,-0.006110
ZEUS,2020-03-30,8.810000,9.760000,8.700000,9.680000,9.680000,73700.0,ZEUS.csv,2020,8.700000,0.112644,1.060000,0.003339


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

It wasn't necessary to convert to pandas because dask can handle the rolling calculation 
But because the data set is small, pandas can do the operation and it will end up faster
I don't think it is better to do it in Dask given the simple rolling calculation

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.